# 🧠 Custom AI Model Training Pipeline

This notebook provides a complete pipeline for training your own AI coding assistant model.

## Training Options:
- 🔥 **Fine-tune CodeLlama**: Recommended for coding tasks
- 🌟 **Fine-tune StarCoder**: Great for code generation
- 🚀 **Fine-tune Llama 2**: General purpose with coding capabilities
- 🛠️ **Custom Training**: Train from scratch (advanced)

## Features:
- ✅ Data collection and preprocessing
- ✅ Model fine-tuning with LoRA/QLoRA
- ✅ Training monitoring and checkpointing
- ✅ Model evaluation and testing
- ✅ Integration with your existing API

## Requirements:
- GPU runtime (T4 minimum, V100/A100 recommended)
- Google Drive for data storage
- Hugging Face account (optional)


In [ ]:
# 📦 Install Training Dependencies

# Option 1: Minimal Setup (Recommended for beginners)
# Upload requirements_minimal.txt to your Colab session, then run:
# !pip install -r requirements_minimal.txt

# Option 2: Manual Installation (if you prefer individual packages)
!pip install transformers==4.35.0
!pip install datasets==2.14.0
!pip install peft==0.6.0
!pip install bitsandbytes==0.41.1
!pip install accelerate==0.24.0
!pip install wandb
!pip install torch==2.1.0
!pip install trl==0.7.4

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import torch
import json
import pandas as pd
from pathlib import Path

# Set up project directories
project_path = '/content/drive/MyDrive/ai-coding-assistant'
training_path = f'{project_path}/training'
data_path = f'{training_path}/data'
models_path = f'{training_path}/models'
logs_path = f'{training_path}/logs'

for path in [training_path, data_path, models_path, logs_path]:
    os.makedirs(path, exist_ok=True)

print(f'✅ Training environment setup complete')
print(f'🔥 GPU Available: {torch.cuda.is_available()}')
print(f'💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB' if torch.cuda.is_available() else 'No GPU')

In [ ]:
# 📊 Data Collection and Preparation

from datasets import Dataset, DatasetDict, load_dataset
import requests
from typing import List, Dict

class CodeDataCollector:
    def __init__(self, data_path: str):
        self.data_path = data_path
        self.datasets = []
    
    def collect_github_data(self, languages: List[str] = ['python', 'javascript', 'java']):
        """Collect code data from GitHub repositories"""
        print("📥 Collecting GitHub data...")
        
        # Use the Stack dataset (filtered GitHub code)
        for lang in languages:
            try:
                print(f"  Downloading {lang} code samples...")
                dataset = load_dataset(
                    "bigcode/the-stack-dedup",
                    data_dir=f"data/{lang}",
                    split="train",
                    streaming=True
                )
                
                # Take first 1000 samples for demo
                samples = []
                for i, sample in enumerate(dataset):
                    if i >= 1000:
                        break
                    samples.append({
                        'content': sample['content'],
                        'language': lang,
                        'source': 'github'
                    })
                
                self.datasets.extend(samples)
                print(f"  ✅ Collected {len(samples)} {lang} samples")
                
            except Exception as e:
                print(f"  ❌ Error collecting {lang} data: {e}")
    
    def collect_stackoverflow_data(self):
        """Collect Q&A data from Stack Overflow"""
        print("📥 Collecting Stack Overflow data...")
        
        try:
            # Use a curated dataset of Stack Overflow questions
            dataset = load_dataset("koutch/stackoverflow_python", split="train")
            
            # Take first 500 samples
            for i, sample in enumerate(dataset):
                if i >= 500:
                    break
                
                self.datasets.append({
                    'content': f"Question: {sample['question']}\n\nAnswer: {sample['answer']}",
                    'language': 'python',
                    'source': 'stackoverflow'
                })
            
            print(f"  ✅ Collected {min(500, len(dataset))} Stack Overflow samples")
            
        except Exception as e:
            print(f"  ❌ Error collecting Stack Overflow data: {e}")
    
    def create_instruction_dataset(self):
        """Convert collected data to instruction format"""
        print("🔄 Converting to instruction format...")
        
        instruction_data = []
        
        for sample in self.datasets:
            if sample['source'] == 'github':
                # Create code explanation tasks
                instruction_data.append({
                    'instruction': f"Explain this {sample['language']} code:",
                    'input': sample['content'][:500],  # Truncate for training
                    'output': f"This {sample['language']} code demonstrates programming concepts and functionality.",
                    'language': sample['language']
                })
                
                # Create code completion tasks
                if len(sample['content']) > 100:
                    split_point = len(sample['content']) // 2
                    instruction_data.append({
                        'instruction': f"Complete this {sample['language']} code:",
                        'input': sample['content'][:split_point],
                        'output': sample['content'][split_point:split_point+200],
                        'language': sample['language']
                    })
            
            elif sample['source'] == 'stackoverflow':
                instruction_data.append({
                    'instruction': "Answer this programming question:",
                    'input': sample['content'],
                    'output': "Here's a solution to your programming question.",
                    'language': sample['language']
                })
        
        return instruction_data
    
    def save_dataset(self, instruction_data: List[Dict]):
        """Save the prepared dataset"""
        dataset_file = f'{self.data_path}/training_dataset.json'
        
        with open(dataset_file, 'w') as f:
            json.dump(instruction_data, f, indent=2)
        
        print(f"💾 Dataset saved to {dataset_file}")
        print(f"📊 Total samples: {len(instruction_data)}")
        
        return dataset_file

# Initialize data collector
collector = CodeDataCollector(data_path)
print('✅ Data collector initialized')

In [ ]:
# 🚀 Start Data Collection

# Collect data from various sources
collector.collect_github_data(['python', 'javascript'])  # Start with 2 languages
collector.collect_stackoverflow_data()

# Convert to instruction format
instruction_data = collector.create_instruction_dataset()

# Save the dataset
dataset_file = collector.save_dataset(instruction_data)

print(f'\n🎉 Data collection complete!')
print(f'📁 Dataset location: {dataset_file}')
print(f'📊 Total training samples: {len(instruction_data)}')

In [ ]:
# 🤖 Setup Model Training

from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
import torch

class ModelTrainer:
    def __init__(self, model_name: str = "codellama/CodeLlama-7b-hf"):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.training_args = None
    
    def load_model_and_tokenizer(self):
        """Load the base model and tokenizer"""
        print(f"📥 Loading model: {self.model_name}")
        
        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        # Load model with 4-bit quantization for memory efficiency
        from transformers import BitsAndBytesConfig
        
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True
        )
        
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto",
            trust_remote_code=True
        )
        
        print("✅ Model and tokenizer loaded")
    
    def setup_lora(self):
        """Setup LoRA for efficient fine-tuning"""
        print("🔧 Setting up LoRA configuration...")
        
        lora_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            r=16,  # Rank
            lora_alpha=32,
            lora_dropout=0.1,
            target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
            bias="none"
        )
        
        self.model = get_peft_model(self.model, lora_config)
        self.model.print_trainable_parameters()
        
        print("✅ LoRA setup complete")
    
    def prepare_dataset(self, dataset_file: str):
        """Prepare the dataset for training"""
        print("📊 Preparing dataset...")
        
        # Load the dataset
        with open(dataset_file, 'r') as f:
            data = json.load(f)
        
        # Format for training
        formatted_data = []
        for sample in data:
            text = f"### Instruction: {sample['instruction']}\n### Input: {sample['input']}\n### Output: {sample['output']}"
            formatted_data.append({'text': text})
        
        # Create dataset
        dataset = Dataset.from_list(formatted_data)
        
        # Tokenize
        def tokenize_function(examples):
            return self.tokenizer(
                examples['text'],
                truncation=True,
                padding=True,
                max_length=512
            )
        
        tokenized_dataset = dataset.map(tokenize_function, batched=True)
        
        # Split into train/validation
        train_test_split = tokenized_dataset.train_test_split(test_size=0.1)
        
        print(f"✅ Dataset prepared - Train: {len(train_test_split['train'])}, Val: {len(train_test_split['test'])}")
        
        return train_test_split
    
    def setup_training_args(self):
        """Setup training arguments"""
        self.training_args = TrainingArguments(
            output_dir=f'{models_path}/checkpoints',
            num_train_epochs=3,
            per_device_train_batch_size=4,
            per_device_eval_batch_size=4,
            gradient_accumulation_steps=4,
            warmup_steps=100,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=10,
            evaluation_strategy="steps",
            eval_steps=100,
            save_steps=500,
            save_total_limit=3,
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            report_to="none"  # Disable wandb for now
        )
        
        print("✅ Training arguments configured")

# Initialize trainer
trainer = ModelTrainer("codellama/CodeLlama-7b-hf")
print('✅ Model trainer initialized')

In [ ]:
# 🚀 Start Model Training

# Load model and tokenizer
trainer.load_model_and_tokenizer()

# Setup LoRA
trainer.setup_lora()

# Prepare dataset
train_dataset = trainer.prepare_dataset(dataset_file)

# Setup training arguments
trainer.setup_training_args()

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=trainer.tokenizer,
    mlm=False
)

# Create Trainer
huggingface_trainer = Trainer(
    model=trainer.model,
    args=trainer.training_args,
    train_dataset=train_dataset['train'],
    eval_dataset=train_dataset['test'],
    data_collator=data_collator,
    tokenizer=trainer.tokenizer
)

print("🎯 Starting training...")
print("⏱️  This will take several hours depending on your GPU")

# Start training
huggingface_trainer.train()

print("🎉 Training completed!")

In [ ]:
# 💾 Save the Trained Model

# Save the model
model_save_path = f'{models_path}/fine_tuned_codellama'

# Save the LoRA adapter
trainer.model.save_pretrained(model_save_path)
trainer.tokenizer.save_pretrained(model_save_path)

print(f"✅ Model saved to: {model_save_path}")

# Save training metadata
metadata = {
    'base_model': trainer.model_name,
    'training_samples': len(instruction_data),
    'training_epochs': trainer.training_args.num_train_epochs,
    'learning_rate': trainer.training_args.learning_rate,
    'model_path': model_save_path,
    'training_date': pd.Timestamp.now().isoformat()
}

with open(f'{model_save_path}/training_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("📋 Training metadata saved")
print("\n🎊 Your custom AI coding assistant model is ready!")

In [ ]:
# 🧪 Test the Trained Model

from peft import PeftModel

def test_model(prompt: str, max_length: int = 200):
    """Test the trained model with a prompt"""
    
    # Format the prompt
    formatted_prompt = f"### Instruction: {prompt}\n### Input: \n### Output: "
    
    # Tokenize
    inputs = trainer.tokenizer(formatted_prompt, return_tensors="pt").to(trainer.model.device)
    
    # Generate
    with torch.no_grad():
        outputs = trainer.model.generate(
            **inputs,
            max_length=max_length,
            temperature=0.7,
            do_sample=True,
            pad_token_id=trainer.tokenizer.eos_token_id
        )
    
    # Decode
    response = trainer.tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the generated part
    generated = response[len(formatted_prompt):].strip()
    
    return generated

# Test with various prompts
test_prompts = [
    "Write a Python function to calculate factorial",
    "Create a JavaScript function to reverse a string",
    "Explain what this code does: for i in range(10): print(i)",
    "Complete this Python code: def fibonacci(n):"
]

print("🧪 Testing the trained model...")
print("=" * 50)

for i, prompt in enumerate(test_prompts, 1):
    print(f"\nTest {i}: {prompt}")
    print("-" * 30)
    
    try:
        response = test_model(prompt)
        print(f"Response: {response}")
    except Exception as e:
        print(f"Error: {e}")
    
    print("=" * 50)

print("✅ Model testing complete!")

In [ ]:
# 🔧 Create Inference Service for Integration

inference_service_code = '''
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import logging

logger = logging.getLogger(__name__)

class CustomModelService:
    def __init__(self, model_path: str, base_model_name: str = "codellama/CodeLlama-7b-hf"):
        self.model_path = model_path
        self.base_model_name = base_model_name
        self.tokenizer = None
        self.model = None
        self.load_model()
    
    def load_model(self):
        """Load the fine-tuned model"""
        try:
            logger.info(f"Loading custom model from {self.model_path}")
            
            # Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_path)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            
            # Load base model
            base_model = AutoModelForCausalLM.from_pretrained(
                self.base_model_name,
                torch_dtype=torch.float16,
                device_map="auto"
            )
            
            # Load LoRA adapter
            self.model = PeftModel.from_pretrained(base_model, self.model_path)
            
            logger.info("Custom model loaded successfully")
            
        except Exception as e:
            logger.error(f"Error loading custom model: {e}")
            raise
    
    async def generate_code(self, prompt: str, language: str = "python") -> str:
        """Generate code using the custom model"""
        try:
            formatted_prompt = f"### Instruction: Generate {language} code for: {prompt}\n### Input: \n### Output: "
            
            inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.model.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_length=512,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            generated = response[len(formatted_prompt):].strip()
            
            return generated
            
        except Exception as e:
            logger.error(f"Error generating code: {e}")
            return f"Error generating code: {str(e)}"
    
    async def suggest_completion(self, code: str, cursor_position: int) -> str:
        """Suggest code completion"""
        try:
            formatted_prompt = f"### Instruction: Complete this code\n### Input: {code}\n### Output: "
            
            inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.model.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_length=256,
                    temperature=0.3,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            generated = response[len(formatted_prompt):].strip()
            
            return generated
            
        except Exception as e:
            logger.error(f"Error suggesting completion: {e}")
            return f"Error: {str(e)}"
    
    async def chat_with_context(self, message: str, context=None) -> str:
        """Chat with context using the custom model"""
        try:
            formatted_prompt = f"### Instruction: Answer this programming question\n### Input: {message}\n### Output: "
            
            inputs = self.tokenizer(formatted_prompt, return_tensors="pt").to(self.model.device)
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_length=512,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id
                )
            
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            generated = response[len(formatted_prompt):].strip()
            
            return generated
            
        except Exception as e:
            logger.error(f"Error in chat: {e}")
            return f"Error: {str(e)}"
'''

# Save the inference service
with open(f'{models_path}/custom_model_service.py', 'w') as f:
    f.write(inference_service_code.strip())

print(f"✅ Custom model inference service created at: {models_path}/custom_model_service.py")
print("\n🔧 Integration Instructions:")
print("1. Copy custom_model_service.py to your backend/app/services/ directory")
print("2. Update your AI service to use the custom model")
print("3. Ensure the model files are accessible from your backend")
print("4. Test the integration with your existing API endpoints")

## 🎉 Training Complete!

Your custom AI coding assistant model has been successfully trained and is ready for use.

### 📊 What You've Accomplished:

1. **Data Collection**: Gathered coding data from GitHub and Stack Overflow
2. **Data Preprocessing**: Converted raw data to instruction format
3. **Model Fine-tuning**: Fine-tuned CodeLlama with LoRA for efficiency
4. **Model Testing**: Validated the model's performance
5. **Integration Service**: Created inference service for your API

### 🔧 Next Steps:

#### Option 1: Use in Colab
- Continue using the model directly in this Colab environment
- Perfect for experimentation and testing

#### Option 2: Download and Deploy Locally
- Download the model files from Google Drive
- Set up local inference server
- Integrate with your existing backend

#### Option 3: Hybrid Approach
- Use custom model for specific tasks
- Fall back to OpenAI API when needed
- Best of both worlds

### 📁 Files Created:

- `training_dataset.json`: Your training data
- `fine_tuned_codellama/`: Your trained model
- `custom_model_service.py`: Integration service
- `training_metadata.json`: Training information

### 🚀 Performance Tips:

1. **More Data**: Collect more domain-specific data for better performance
2. **Longer Training**: Increase epochs for better convergence
3. **Hyperparameter Tuning**: Experiment with learning rates and batch sizes
4. **Model Selection**: Try different base models (StarCoder, Llama 2, etc.)

### 💡 Advanced Features:

- **Multi-language Support**: Train on multiple programming languages
- **Code Review**: Fine-tune for code review and bug detection
- **Documentation**: Train for automatic documentation generation
- **Testing**: Generate unit tests automatically

**Congratulations! You now have your own AI coding assistant! 🎊**
